# Automate Event Planning

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')
from dotenv import load_dotenv,find_dotenv

load_dotenv(find_dotenv())
import os   

from phoenix.otel import register

# configure the Phoenix tracer
tracer_provider = register(
  project_name="my-crewai-app", # Default is 'default'
  auto_instrument=True # Auto-instrument your app based on installed OI dependencies
)

Overriding of current TracerProvider is not allowed


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: my-crewai-app
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [ ]:
#Groq
os.environ["OPENAI_API_KEY"] = os.environ["GROQ_API_KEY"] 
os.environ["OPENAI_API_BASE"] = os.environ["GROQ_API_BASE"] 
os.environ["OPENAI_MODEL_NAME"] = "meta-llama/llama-4-scout-17b-16e-instruct"

In [2]:
#Gemini API
# If you are using Gemini API, set the OpenAI API key to the Gemini API key.
os.environ["OPENAI_API_KEY"] = os.environ["GEMINI_API_KEY"] 
os.environ["OPENAI_MODEL_NAME"] = 'gemini/gemini-1.5-flash'

In [2]:
#Openrouter
os.environ["OPENAI_API_KEY"] = os.environ["OPENROUTER_API_KEY"] 
os.environ["OPENAI_API_BASE"] = os.environ["OPENROUTER_API_BASE"] 
os.environ["OPENAI_MODEL_NAME"] = "mistralai/mistral-small-24b-instruct-2501:free"

- Import libraries, APIs and LLM

In [3]:
from crewai import Agent, Crew, Task
from langchain_community.tools import DuckDuckGoSearchRun
from crewai.tools import BaseTool

In [4]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from pydantic import Field

In [5]:
search = DuckDuckGoSearchAPIWrapper()

class SearchTool(BaseTool):
    name: str = "Search"
    description: str = "Useful for search-based queries. Use this to find current information about markets, companies, and trends."
    search: DuckDuckGoSearchAPIWrapper = Field(default_factory=DuckDuckGoSearchAPIWrapper)

    def _run(self, query: str) -> str:
        """Execute the search query and return results"""
        try:
            return self.search.run(query)
        except Exception as e:
            return f"Error performing search: {str(e)}"

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

## crewAI Tools

In [6]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
# search_tool = SerperDevTool()
search_tool = SearchTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [7]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [8]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [9]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [10]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [11]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [12]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [13]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=False,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Creating the Crew

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [14]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [15]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

**Note 1**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

**Note 2**: 
- Since you set `human_input=True` for some tasks, the execution will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

In [16]:
result = event_management_crew.kickoff(inputs=event_details)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: f8aff0d6-2582-463b-8611-7616f632889b                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 8289a396-8c50-41d5-bd03-6cbfb4ef494c
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 8289a396-8c50-41d5-bd03-6cbfb4ef494c
       Status: Executing Task...
    └── 🤖 Agent: Venue Coordinator
            Status: In Progress

# Agent: Venue Coordinator
## Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


🤖 Agent: Venue Coordinator
    Status: In Progress
└── 🧠 Thinking...

...

🤖 Agent: Marketing and Communications Agent
    Status: In Progress

🚀 Crew: crew
├── 📋 Task: 8289a396-8c50-41d5-bd03-6cbfb4ef494c
│      Assigned to: Venue Coordinator
│      Status: ✅ Completed
│   └── 🤖 Agent: Venue Coordinator
│           Status: ✅ Completed
├── 📋 Task: 43f4f41f-4bd7-4d92-89c3-b61e06e96511
│      Assigned to: Logistics Manager
│      Status: ✅ Completed
│   └── 🤖 Agent: Logistics Manager
│           Status: ✅ Completed
└── 📋 Task: 65b37ee1-0d66-46d8-8b32-ac3f51e94fc8
       Status: Executing Task...
    └── 🤖 Agent: Marketing and Communications Agent
            Status: In Progress

╭────────────────────────────────────────────────── Tool Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  Tool Usage Failed                                                                                              │
│  Name: Search                                                                                                   │
│  Error: Arguments validation failed: 1 validation error for SearchToolSchema                                    │
│  query                                                                                                          │
│    Input should be a valid string [type=string_type, input_value={'description': None, 'type': 'str'},          │
│  input_type=dict]                                                                                               │
│      For further information visit https://errors.pydantic.dev/2.11/v/string_type                               │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 1 validation error for SearchToolSchema
query
  Input should be a valid string [type=string_type, input_value={'description': None, 'type': 'str'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type.
 Tool Search accepts these inputs: Tool Name: Search
Tool Arguments: {'query': {'description': None, 'type': 'str'}}
Tool Description: Useful for search-based queries. Use this to find current information about markets, companies, and trends.



# Agent: Marketing and Communications Agent
## Thought: tool_code
Thought:To effectively promote the Tech Innovation Conference and engage at least 500 potential attendees, I need to understand the conference details and identify suitable marketing channels.  I'll start by searching for information on effective marketing strategies for tech conferences.
## Using tool: Search
## Tool Input: 
"{\"quer

KeyboardInterrupt: 

- Display the generated `venue_details.json` file.

In [ ]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

- Display the generated `marketing_report.md` file.

**Note**: After `kickoff` execution has successfully ran, wait an extra 45 seconds for the `marketing_report.md` file to be generated. If you try to run the code below before the file has been generated, your output would look like:

```
marketing_report.md
```

If you see this output, wait some more and than try again.

In [ ]:
from IPython.display import Markdown
Markdown("marketing_report.md")